In [1]:
import tools
from tools.imports import *
from tools.document_tools import GoogleDocWriteTool
from tools.file_mgmt_tools import MoveFileTool, CreateFolderTool, FolderMovementTool, FileOrganizerTool, ImprovedSearchTool
from tools.miscellaneous_mgmt import GoogleDriveUploadTool, GmailSendPdfTool, GoogleSheetsUpdateTool, GoogleSheetsCreateTool
from tools.initialize_groq import init_groq
from dotenv import load_dotenv

In [2]:
client, llm = init_groq()

In [3]:
credentials_path = os.getenv('CREDENTIALS_PATH')
my_tools = []
my_tools.extend(
    
    [GoogleDocWriteTool(credentials_path),
    GoogleSheetsUpdateTool(credentials_path),
    GoogleSheetsCreateTool(credentials_path),
    GmailSendPdfTool(credentials_path),
    MoveFileTool(credentials_path),
    CreateFolderTool(credentials_path),
    FolderMovementTool(credentials_path),
    FileOrganizerTool(credentials_path),
    ImprovedSearchTool(credentials_path),
    ]
)

In [4]:
chat_history = []

In [5]:
my_tools

[GoogleDocWriteTool(credentials_path='paths/client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json', creds=<google.oauth2.credentials.Credentials object at 0x000001D1AE6AED20>, docs_service=<googleapiclient.discovery.Resource object at 0x000001D1AE63E570>, drive_service=<googleapiclient.discovery.Resource object at 0x000001D1AE6DA960>),
 GoogleSheetsUpdateTool(credentials_path='paths/client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json', creds=<google.oauth2.credentials.Credentials object at 0x000001D1AE6DAD50>, service=<googleapiclient.discovery.Resource object at 0x000001D1AE797890>),
 GoogleSheetsCreateTool(credentials_path='paths/client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json', creds=<google.oauth2.credentials.Credentials object at 0x000001D1AE797AA0>, service=<googleapiclient.discovery.Resource object at 0x000001D1AE878620>),
 GmailSendPdfTool(credentials_path='

In [6]:
import tools.initialize_groq
import random
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType, AgentExecutor
from google.cloud import texttospeech
from langchain.agents import load_agent


# Define the template
template = """
YOU ARE A VERY ADVANCED DOCUMENT MANAGER WHO USES GOOGLE DRIVE FOR DOCUMENT MANAGEMENT.
WHEN USER INDICATES THEY WANT TO MOVE SOMETHING INTO GOOGLE DRIVE OR MY DRIVE, YOU PASS IN 'ROOT'!!!!! OKAY!!!!!
RESPOND IN A CLEAR CUT MANNER.
DO NOT SAY THINGS LIKE - 'here is the response' and the like. OKAY!!?!??
YOU SHALL NOT INDICATE ANY TOOL USE UNTIL YOU KNOW YOU HAVE EVERYTHING YOU NEED.
DO NOT ASSUME USER WANTS TO DO ANYTHING AT ALL UNLESS YOU ARE 100% SURE!!!!! UNDERSTAND??????!!!!!! OR ELSE I WILL BECOME ANGRY
BE REALLY CAREFUL WITH FILE AND FOLDER ID'S! ANY WRONG ID'S WILL RESULT IN FAILURE OF OPERATIONS

If user tells you to do something that is not one of these tools/operations,\
      you kindly say that you don't have access to that functionality.

"""



# User credentials
credentials = {
    "name": "Gautham Ramachandran",
    "email": "sriramnallani35@gmail.com",
    "recemail": "gauthamramachandran3@gmail.com",
    "phone": "5715996302"
}


def synthesize_speech(text, output_file):
    print('THE TEXT: ', text)
    idx = text.rfind('@')
    text = text[idx + 1:].strip()
    print('MODIFIED RESPONSE = ', text)
    tts_service_acct_path = os.getenv('SERVICE_ACCOUNT_PATH')
    client = texttospeech.TextToSpeechClient.from_service_account_file(tts_service_acct_path)

    
    input_text = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        ssml_gender=texttospeech.SsmlVoiceGender.MALE
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = client.synthesize_speech(
        input=input_text,
        voice=voice,
        audio_config=audio_config
    )
    with open(output_file, "wb") as out:
        out.write(response.audio_content)
    print(f'Audio content written to "{output_file}"')

from langchain.tools import HumanInputRun
my_tools.append([HumanInputRun])

agent = None
# Define a function to run the agent and handle the response
def run_agent(input_text):
    global chat_history
    input_text += "ALWAYS GENERATE THE GOOGLE DOC CONTENT YOURSELF! NEVER SPEAK TO HUMAN UNLESS INSTRUCTED. \
            DO ONLY WHAT IS SAID IN INSTRUCTIONS, AND FORMAT YOUR RESPONSES ONLY AS SAID IN INSTRUCTIONS OR ELSE LIFE WILL END AS WE KNOW IT. \
            DO EVERYTHING SAID HERE. NOT EVEN ONE THING SHALL BE LEFT INCOMPLETE.\
            DO NOT CREATE FOLDERS WHEN ORGANIZING FILES.\
            UNLESS USER TELLS YOU TO WRITE SOMETHING IN GOOGLE DOC DO NOT PASS ANYTHING INTO 'input_text' PARAMETER! \
            ALSO, WHEN MOVING A FOLDER INTO ANOTHER FOLDER, ALWAYS USE FOLDER MOVEMENT TOOL\n" + template + "\n"
    
    input_text += ("\n\n Here is extra info you will need: \nCredentials:\n" + str(credentials) + "\n"
                   + "Path to file on local system:\n\nTHE CHAT HISTORY: \n"+str(chat_history))
    print(input_text)
    llm.groq_api_key = random.choice(tools.initialize_groq.api_keys)

    agent = initialize_agent(
        tools=my_tools,
        llm=llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        handle_parsing_errors=True,
        verbose=True,
        max_iterations=1000,
        return_intermediate_steps=True
    )
    
    result = agent.invoke({
        "input": input_text,
    })
    chat_history.append({"input": input_text, "response": result})
    mystr = (str(result['intermediate_steps']) + "\n" + str(result['output']))
    
    
    synthesize_speech(client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content":"please sanitize this input so that someone can speak it. START THE SPEAKABLE INPUT WITH '@' symbol: " + mystr
            }
        ],
        model='llama3-70b-8192',
    ).choices[0].message.content, "intermediateoutput.mp3"
    )

    return result

user_input = f""" 
can you create a google sheet called appointments and fill in 5 rows?
""" 
result = run_agent(user_input)
print(result['output'])


 
can you create a google sheet called appointments and fill in 5 rows?
ALWAYS GENERATE THE GOOGLE DOC CONTENT YOURSELF! NEVER SPEAK TO HUMAN UNLESS INSTRUCTED.             DO ONLY WHAT IS SAID IN INSTRUCTIONS, AND FORMAT YOUR RESPONSES ONLY AS SAID IN INSTRUCTIONS OR ELSE LIFE WILL END AS WE KNOW IT.             DO EVERYTHING SAID HERE. NOT EVEN ONE THING SHALL BE LEFT INCOMPLETE.            DO NOT CREATE FOLDERS WHEN ORGANIZING FILES.            UNLESS USER TELLS YOU TO WRITE SOMETHING IN GOOGLE DOC DO NOT PASS ANYTHING INTO 'input_text' PARAMETER!             ALSO, WHEN MOVING A FOLDER INTO ANOTHER FOLDER, ALWAYS USE FOLDER MOVEMENT TOOL

YOU ARE A VERY ADVANCED DOCUMENT MANAGER WHO USES GOOGLE DRIVE FOR DOCUMENT MANAGEMENT.
WHEN USER INDICATES THEY WANT TO MOVE SOMETHING INTO GOOGLE DRIVE OR MY DRIVE, YOU PASS IN 'ROOT'!!!!! OKAY!!!!!
RESPOND IN A CLEAR CUT MANNER.
DO NOT SAY THINGS LIKE - 'here is the response' and the like. OKAY!!?!??
YOU SHALL NOT INDICATE ANY TOOL USE UNTIL YOU

c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


AttributeError: 'list' object has no attribute 'args'

In [ ]:
result

In [ ]:
import os
import random
import asyncio
import pyaudio
import wave
import threading
import keyboard  # Add this import
from google.cloud import texttospeech
from langchain.prompts import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate, 
    MessagesPlaceholder, 
    PromptTemplate
)
from langchain_core.messages import SystemMessage
from langchain.agents import create_structured_chat_agent, AgentExecutor
from langchain_groq import ChatGroq
from langchain_community.tools import HumanInputRun
import tools.initialize_groq
import typing
import langchain_core
from pydub import AudioSegment
from pydub.playback import play
import whisper

prompt = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
    input_types={
        'chat_history': typing.List[
            typing.Union[
                langchain_core.messages.ai.AIMessage, 
                langchain_core.messages.human.HumanMessage, 
                langchain_core.messages.chat.ChatMessage, 
                langchain_core.messages.system.SystemMessage, 
                langchain_core.messages.function.FunctionMessage, 
                langchain_core.messages.tool.ToolMessage
            ]
        ]
    },
    metadata={
        'lc_hub_owner': 'hwchase17',
        'lc_hub_repo': 'structured-chat-agent',
        'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'
    },
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['tool_names', 'tools'],
                template=(
                    'You are a conversational LLM, and you ARE ALSO STRAIGHTFORWARD.'
                    'YOU GIVE THE USER THE ANSWERS THAT THEY WANT IN A STRAIGHTFORWARD WAY. YOU NEVER ASK FOR CLARIFICATION OR WASTE THE USERS TIME BY DOING SO!'
                    'Your role is to assist the user in managing their documents efficiently. '
                    'You have access to the following tools:\n\n{tools}\n\n'
                    'Use a JSON blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\n'
                    'Valid "action" values: "Final Answer" or {tool_names}\n\n'
                    'Provide only ONE action per $JSON_BLOB, as shown:\n\n'
                    '```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\n'
                    'Follow this format:\n\n'
                    'Question: input question to answer\n'
                    'Thought: consider previous and subsequent steps\n'
                    'Action:\n```\n$JSON_BLOB\n```\n'
                    'Observation: action result\n... (repeat Thought/Action/Observation N times)\n'
                    'Thought: I know what to respond\n'
                    'Action:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\n'
                    'Begin! Remember to ALWAYS respond with a valid JSON blob of a single action. '
                    'Use tools if necessary and respond directly if appropriate. '
                    'Ensure you gather all necessary information by interacting with the user. '
                    'Format is Action:```$JSON_BLOB```then Observation.'
                )
            )
        ),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['agent_scratchpad', 'input'],
                template='{input}\n\n{agent_scratchpad}\n(reminder to respond in a JSON blob no matter what)'
            )
        )
    ]
)



In [ ]:
import os
import random
import asyncio
import pyaudio
import wave
import threading
import keyboard  # Add this import
from google.cloud import texttospeech
from langchain.prompts import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate, 
    MessagesPlaceholder, 
    PromptTemplate
)
from langchain_core.messages import SystemMessage
from langchain.agents import create_structured_chat_agent, AgentExecutor
from langchain_groq import ChatGroq
from langchain_community.tools import HumanInputRun
import tools.initialize_groq
import typing
import langchain_core
from pydub import AudioSegment
from pydub.playback import play
import whisper

prompt = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
    input_types={
        'chat_history': typing.List[
            typing.Union[
                langchain_core.messages.ai.AIMessage, 
                langchain_core.messages.human.HumanMessage, 
                langchain_core.messages.chat.ChatMessage, 
                langchain_core.messages.system.SystemMessage, 
                langchain_core.messages.function.FunctionMessage, 
                langchain_core.messages.tool.ToolMessage
            ]
        ]
    },
    metadata={
        'lc_hub_owner': 'hwchase17',
        'lc_hub_repo': 'structured-chat-agent',
        'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'
    },
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['tool_names', 'tools'],
                template=(
                    'You are a document management assistant proficient in using GSuite tools. '
                    'Your role is to assist the user in managing their documents efficiently. '
                    'You have access to the following tools:\n\n{tools}\n\n'
                    'Use a JSON blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\n'
                    'Valid "action" values: "Final Answer" or {tool_names}\n\n'
                    'Provide only ONE action per $JSON_BLOB, as shown:\n\n'
                    '```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\n'
                    'Follow this format:\n\n'
                    'Question: input question to answer\n'
                    'Thought: consider previous and subsequent steps\n'
                    'Action:\n```\n$JSON_BLOB\n```\n'
                    'Observation: action result\n... (repeat Thought/Action/Observation N times)\n'
                    'Thought: I know what to respond\n'
                    'Action:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\n'
                    'Begin! Remember to ALWAYS respond with a valid JSON blob of a single action. '
                    'Use tools if necessary and respond directly if appropriate. '
                    'Ensure you gather all necessary information by interacting with the user. '
                    'Format is Action:```$JSON_BLOB```then Observation.'
                )
            )
        ),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['agent_scratchpad', 'input'],
                template='{input}\n\n{agent_scratchpad}\n(reminder to respond in a JSON blob no matter what)'
            )
        )
    ]
)


# Initialize global variables
chat_history = []
is_recording = False  # Ensure this is defined
frames = []  # Ensure this is defined


In [ ]:
# Initialize Google TTS client
tts_client = texttospeech.TextToSpeechClient.from_service_account_file(os.getenv('SERVICE_ACCOUNT_PATH'))

def synthesize_speech(text, output_file):
    idx = text.rfind('@')
    text = text[idx + 1:].strip()
    input_text = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(language_code="en-US", ssml_gender=texttospeech.SsmlVoiceGender.MALE)
    audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)
    response = tts_client.synthesize_speech(input=input_text, voice=voice, audio_config=audio_config)
    with open(output_file, "wb") as out:
        out.write(response.audio_content)
    print(f'Audio content written to "{output_file}"')

def play_audio(file):
    audio = AudioSegment.from_mp3(file)
    play(audio)

def text_to_speech_prompt(prompt):
    output_file = 'test_output.mp3'
    synthesize_speech(prompt, output_file=output_file)
    play_audio(output_file)


In [ ]:
from langchain import hub

async def human_input_run():
    global chat_history
    
    user_input = speech_to_text_input()
    human_tool = HumanInputRun(prompt_func=text_to_speech_prompt, input_func=speech_to_text_input)
    human_agent = create_structured_chat_agent(llm=llm, tools=[human_tool], prompt=hub.pull("hwchase17/structured-chat-agent"))
    agent_executor = AgentExecutor(
        agent=human_agent,
        tools=[human_tool],
        verbose=True,
        handle_parsing_errors=True,
        return_intermediate_steps=True,
    )
    result = await agent_executor.ainvoke({"input": user_input})
    chat_history.append({"input": user_input, "response": result})
    output_file = "final_output.mp3"
    synthesize_speech(result['output'], output_file)
    play_audio(output_file)
    return result['output']  # Return the output



async def document_manager_run(input_text):
    credentials = {
        "name": "Gautham Ramachandran",
        "email": "sriramnallani35@gmail.com",
        "recemail": "gauthamramachandran3@gmail.com",
        "phone": "5715996302"
    }
    global chat_history

    input_text += "\n\n Here is extra info you will need: \nCredentials:\n" + str(credentials) + "\nTHE CHAT HISTORY: \n" + str(chat_history)
    print(input_text)

    # Set the Groq API key randomly
    llm.groq_api_key = random.choice(tools.initialize_groq.api_keys)

    search_agent = create_structured_chat_agent(llm, my_tools, prompt)
    agent_executor = AgentExecutor(
        agent=search_agent,
        tools=my_tools,
        verbose=True,
        handle_parsing_errors=True,
        return_intermediate_steps=True,
    )
    
    result = await agent_executor.ainvoke({"input": input_text})
    chat_history.append({"input": input_text, "response": result})
    mystr = (str(result['intermediate_steps']) + "\n" + str(result['output']))

    for step in result['intermediate_steps']:
        output_file = f"intermediate_output_{step['index']}.mp3"
        synthesize_speech(step['observation'], output_file)
        play_audio(output_file)

    final_response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "please sanitize this input so that someone can speak it. START THE SPEAKABLE INPUT WITH '@' symbol: " + mystr
            }
        ],
        model='llama3-70b-8192',
    ).choices[0].message.content

    output_file = "final_output.mp3"
    synthesize_speech(final_response, output_file)
    play_audio(output_file)

    return result


In [ ]:
# Example usage
user_input = "hello"

# Run the human input agent and document manager agent sequentially
async def main():
    
    await document_manager_run(user_input)  # Use human_output as input


await main()
